<a href="https://colab.research.google.com/github/amitthakur2013/Coursera_Capstone/blob/master/Assignment_capstone_week3_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import all necessary libraries**

In [1]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
print("Import Successful!!")

Import Successful!!


**Get The URL or link to the wikipedia website containing Toronto Neighbourhood**

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

**Make GET request using requests libary and parse the html document using BeautifulSoup library frm bs4 package of python**

In [0]:
response=requests.get(url)
soup=BeautifulSoup(response.text,'html.parser')
table=soup.find('table',{'class':'wikitable sortable'})

**Preprocessing and formatting of the data to make the Data Set**

In [0]:
rows=table.find_all('tr')
columns=[v.text.replace('\n','') for v in rows[0].find_all('th')]

df=pd.DataFrame(columns=columns)

for i in range(1,len(rows)):
  tds=rows[i].find_all('td')
  if(tds[1].text == 'Not assigned'):
    continue
  if(tds[2].text =='Not assigned\n'):
    values=[tds[0].text,tds[1].text,tds[1].text]
  else:
    values=[tds[0].text,tds[1].text,tds[2].text.replace('\n','')]
  df=df.append(pd.Series(values, index=columns), ignore_index=True)

df2=pd.DataFrame(columns=columns)

df2['Postcode']=df['Postcode'].unique()

df2.set_index('Postcode',inplace=True)

df2.fillna("", inplace = True) 

for i in range(0,len(df)):
  x=str(df.iloc[i][0])
  df2.loc[x,'Neighbourhood']=","+str(df.iloc[i][2])+df2.loc[x,'Neighbourhood']

df2['Neighbourhood']=df2['Neighbourhood'].apply(lambda x: x[1:])

df_check=df.copy()
for j,i in enumerate(df_check['Postcode'].values):
  df2.loc[i,'Borough']=df.iloc[j]['Borough']

df2.reset_index(inplace=True)

**Final Dataset**

In [5]:
df2.head()


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


**Import geospatial data to find the latitude and longitude of the neighbourhoods using postal code**

In [16]:
data=pd.read_csv('Geospatial_Coordinates.csv')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [0]:
data.set_index('Postal Code',inplace=True)

In [0]:
data.head()

In [0]:
val1,val2=[],[]
for j,i in enumerate(df2['Postcode'].values.tolist()):
  val1.append(data.loc[i,'Latitude'])
  val2.append(data.loc[i,'Longitude'])
df2['Latitude']=val
df2['Longitude']=val2

**Final dataset**

In [41]:
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
